In [1]:
import pandas as pd
import numpy as np
#载入数据
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data['origin'] = 'train'
test_data['origin'] = 'test'
data = pd.concat([train_data, test_data], ignore_index=True, sort=False)
#重命名列名
data.rename(columns = {'fecha_dato':'date','ncodpers':'user_id','ind_empleado':'employee_type',
                        'pais_residencia':'user_country','sexo':'sex','fecha_alta':'register_date',
                        'ind_nuevo':'new_user_type','antiguedad':'user_seniority','indrel':'user_type',
                        'ult_fec_cli_1t':'last_date_as_primary_user','indrel_1mes':'user_type_at_begin',
                        'tiprel_1mes':'user_relation_at_begin','indresi':'live_country_type','indext':'born_country_type',
                        'conyuemp':'employee_spouse_type','canal_entrada':'user_join_channel','indfall':'alive_type',
                        'tipodom':'address_type','cod_prov':'province_code','nomprov':'province_name',
                        'ind_actividad_cliente':'activity_type','renta':'user_income','segmento':'user_segmentation',
                        'ind_ahor_fin_ult1':'item_1','ind_aval_fin_ult1':'item_2','ind_cco_fin_ult1':'item_3',
                        'ind_cder_fin_ult1':'item_4','ind_cno_fin_ult1':'item_5','ind_ctju_fin_ult1':'item_6',
                        'ind_ctma_fin_ult1':'item_7','ind_ctop_fin_ult1':'item_8','ind_ctpp_fin_ult1':'item_9',
                        'ind_deco_fin_ult1':'item_10','ind_deme_fin_ult1':'item_11','ind_dela_fin_ult1':'item_12',
                        'ind_ecue_fin_ult1':'item_13','ind_fond_fin_ult1':'item_14','ind_hip_fin_ult1':'item_15',
                        'ind_plan_fin_ult1':'item_16','ind_pres_fin_ult1':'item_17','ind_reca_fin_ult1':'item_18',
                        'ind_tjcr_fin_ult1':'item_19','ind_valo_fin_ult1':'item_20','ind_viv_fin_ult1':'item_21',
                        'ind_nomina_ult1':'item_22','ind_nom_pens_ult1':'item_23','ind_recibo_ult1':'item_24'},inplace = True)
#去除一些相关性不太大的特征列
data = data.dropna(axis=0,subset = ['register_date'])
data = data.drop('employee_spouse_type',axis = 1)
data = data.drop('last_date_as_primary_user',axis = 1)
data = data.drop('province_name',axis = 1)
data = data.drop('address_type',axis = 1)
data = data.drop('employee_type',axis = 1)
#对特征列的空值进行补全
data['user_type'] = data['user_type'].replace(np.nan, 2)
data['province_code'] = data['province_code'].replace(np.nan, 99)
data['activity_type'] = data['activity_type'].replace(np.nan, 2)
data['user_country'] = data['user_country'].replace(np.nan, 'unknown')
data['sex'] = data['sex'].replace(np.nan,'unknown')
data['user_seniority'] = data['user_seniority'].replace(np.nan, 0)
data['user_seniority'] = data['user_seniority'].replace('     NA', 0)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1.0', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3.0', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2.0', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4.0', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('P', 5)
data['user_type_at_begin'] = data['user_type_at_begin'].replace(np.nan, 0)
data['user_relation_at_begin'] = data['user_relation_at_begin'].replace(np.nan, 'unknown')
data['live_country_type'] = data['live_country_type'].replace(np.nan, 'unknown')
data['born_country_type'] = data['born_country_type'].replace(np.nan, 'unknown')
data['user_join_channel'] = data['user_join_channel'].replace(np.nan, 'unknown')
data = data.drop('alive_type',axis = 1)
data['user_segmentation'] = data['user_segmentation'].replace(np.nan, 'unknown')
#增加用户使用时长的特征
data['date']=pd.to_datetime(data['date'],format='%Y-%m-%d')
data['register_date']=pd.to_datetime(data['register_date'],format='%Y-%m-%d')
data['use_time'] = (data['date'] - data['register_date']).dt.days
data = data.drop('register_date',axis =1)


D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#提取用户信息特征准备通过xgboost对用户收入进行补全
user_data = data[['user_id','date','user_country','sex','age',
                'user_seniority','user_type','live_country_type','born_country_type',
                'user_join_channel','province_code','activity_type','user_income',
                'user_segmentation','use_time']]
user_data = user_data.sort_values(by=['user_id','date'])
user_data.drop_duplicates(subset = ['user_id'],keep='last',inplace=True)
user_data = user_data.drop('date',axis = 1)
user_data = user_data.reset_index(drop = True)


In [3]:
print(user_data)

       user_id user_country sex age user_seniority  user_type  \
0        16074           ES   V  47             83        1.0   
1        16095           ES   V  74            255        1.0   
2        16117           ES   V  57            256        1.0   
3        16181           ES   H  77            242        1.0   
4        16308           ES   V  47            248        1.0   
5        16339           ES   V  55            254        1.0   
6        16450           ES   V  55            254        1.0   
7        16465           ES   H  55            116        1.0   
8        16565           ES   V  53            253        1.0   
9        16571           ES   V  50            253        1.0   
10       16579           ES   V  75             90        1.0   
11       16630           ES   H  62            252        1.0   
12       16656           ES   V  64            252        1.0   
13       16861           ES   V  46            249        1.0   
14       16972           

In [4]:
#对用户信息特征列进行labelencoder
user_data1 = user_data[['user_id','sex','age','province_code','user_segmentation','user_income']]
from sklearn.preprocessing import LabelEncoder
lbe_sex = LabelEncoder()
user_data1['sex'] = lbe_sex.fit_transform(user_data1['sex'])
lbe_province_code = LabelEncoder()
user_data1['province_code'] = lbe_province_code.fit_transform(user_data1['province_code'])
lbe_user_segmentation = LabelEncoder()
user_data1['user_segmentation'] = lbe_user_segmentation.fit_transform(user_data1['user_segmentation'])
user_data1['age'] = user_data1['age'].astype('int16')
print(user_data1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1     53478.18
1        16095    1   74             27                  0          NaN
2        16117    1   57             27                  1    465589.68
3        16181    0   77             27                  1    134978.88
4        16308    1   47             27                  1    175988.07
5        16339    1   55             27                  0    197736.39
6        16450    1   55             27                  1    202125.27
7        16465    0   55             27                  1     37449.15
8        16565    1   53             27                  1    187198.65
9        16571    1   50             45                  1    210769.11
10       16579    1   75              7                  1    196727.04
11       16630    0   62             27                  1    187577.91
12       16656    1   64             27                  1    15

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [5]:
#将存在收入数据的用户信息作为训练集，不存在收入数据的用户信息作为预测集
user_data_train = user_data1.dropna()
user_data_train['user_income'] = user_data_train['user_income'].astype('int64')
user_data_predict = user_data1[user_data1.user_income.isnull()]

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
6        16450    1   55             27                  1       202125
7        16465    0   55             27                  1        37449
8        16565    1   53             27                  1       187198
9        16571    1   50             45                  1       210769
10       16579    1   75              7                  1       196727
11       16630    0   62             27                  1       187577
12       16656    1   64             27                  1       156336
13       16861    1   46             27                  1      

In [7]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0          NaN
32       17453    0   75             27                  1          NaN
41       17701    1   53             27                  1          NaN
53       18024    1   60             27                  1          NaN
58       18288    1   70             38                  1          NaN
61       18408    0   60             27                  1          NaN
65       18528    1   60             35                  1          NaN
75       18922    0   53             19                  1          NaN
78       19138    1   51             45                  1          NaN
85       19342    1   78              7                  1          NaN
86       19348    1   57             27                  1          NaN
90       19500    0   44             25                  1          NaN
93       19664    0   60              7                  1      

In [8]:
user_data_predict = user_data_predict.drop('user_income',axis =1)

In [9]:
#将训练集中收入偏差均值3个标准差以外的收入数据进行剔除
user_income_mean = user_data_train['user_income'].mean()
print(user_income_mean)
user_income_std = user_data_train['user_income'].std()
print(user_income_std)
user_income_max = int(user_income_mean + 3 * user_income_std)
print(user_income_max)

136586.759798995
320021.55237731023
1096651


In [10]:
user_data_train1 = user_data_train[user_data_train.user_income < user_income_max]
print(user_data_train1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
6        16450    1   55             27                  1       202125
7        16465    0   55             27                  1        37449
8        16565    1   53             27                  1       187198
9        16571    1   50             45                  1       210769
10       16579    1   75              7                  1       196727
11       16630    0   62             27                  1       187577
12       16656    1   64             27                  1       156336
13       16861    1   46             27                  1      

In [11]:
#进行xgboost预测
import xgboost as xgb
from sklearn.model_selection import train_test_split

param = {'boosting_type':'gbdt',
                         'objective' : 'reg:linear', 
                         'eval_metric' : 'rmse',
                         'eta' : 0.01,
                         'max_depth' : 10,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         #'alpha': 0.6,
                         'lambda': 2,
        }
user_data_predict1 = xgb.DMatrix(user_data_predict[['sex','age','province_code','user_segmentation']])
#for name in col_name:
X_train, X_valid, y_train, y_valid = train_test_split(user_data_train1.drop(['user_income','user_id'],axis=1), user_data_train1['user_income']
                                                      , test_size=0.2, random_state=2020)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)

predict = model.predict(user_data_predict1)


[21:09:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[21:09:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:162566.85938	valid-rmse:163331.79688
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 200 rounds.
[25]	train-rmse:139612.42188	valid-rmse:140550.84375
[50]	train-rmse:123574.67969	valid-rmse:124694.32812
[75]	train-rmse:112719.59375	valid-rmse:114049.73438
[100]	train-rmse:105285.11719	valid-rmse:106821.86719
[125]

In [12]:
user_data_predict['user_income'] = predict

In [13]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation    user_income
1        16095    1   74             27                  0  221646.671875
32       17453    0   75             27                  1  155265.656250
41       17701    1   53             27                  1  165207.000000
53       18024    1   60             27                  1  166919.312500
58       18288    1   70             38                  1  135890.796875
61       18408    0   60             27                  1  174498.437500
65       18528    1   60             35                  1  109981.460938
75       18922    0   53             19                  1   80021.265625
78       19138    1   51             45                  1   86606.242188
85       19342    1   78              7                  1  170857.390625
86       19348    1   57             27                  1  207827.812500
90       19500    0   44             25                  1   94032.054688
93       19664    0   60              

In [14]:
user_data_predict['user_income'] = user_data_predict['user_income'].astype('int64')

In [15]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
6        16450    1   55             27                  1       202125
7        16465    0   55             27                  1        37449
8        16565    1   53             27                  1       187198
9        16571    1   50             45                  1       210769
10       16579    1   75              7                  1       196727
11       16630    0   62             27                  1       187577
12       16656    1   64             27                  1       156336
13       16861    1   46             27                  1      

In [16]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0       221646
32       17453    0   75             27                  1       155265
41       17701    1   53             27                  1       165207
53       18024    1   60             27                  1       166919
58       18288    1   70             38                  1       135890
61       18408    0   60             27                  1       174498
65       18528    1   60             35                  1       109981
75       18922    0   53             19                  1        80021
78       19138    1   51             45                  1        86606
85       19342    1   78              7                  1       170857
86       19348    1   57             27                  1       207827
90       19500    0   44             25                  1        94032
93       19664    0   60              7                  1      

In [17]:
user_data_all = pd.concat([user_data_train,user_data_predict],ignore_index = True)

In [18]:
print(user_data_all)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
1        16117    1   57             27                  1       465589
2        16181    0   77             27                  1       134978
3        16308    1   47             27                  1       175988
4        16339    1   55             27                  0       197736
5        16450    1   55             27                  1       202125
6        16465    0   55             27                  1        37449
7        16565    1   53             27                  1       187198
8        16571    1   50             45                  1       210769
9        16579    1   75              7                  1       196727
10       16630    0   62             27                  1       187577
11       16656    1   64             27                  1       156336
12       16861    1   46             27                  1      

In [19]:
print(data)

             date  user_id user_country sex  age  new_user_type  \
0      2015-01-28  1050676           ES   V   31            0.0   
1      2015-01-28  1050662           ES   V   37            0.0   
2      2015-01-28  1050652           ES   H   31            0.0   
3      2015-01-28  1050489           ES   H   23            0.0   
4      2015-01-28  1050490           ES   V   23            0.0   
5      2015-01-28  1050497           ES   H   23            0.0   
6      2015-01-28  1050500           ES   V   24            0.0   
7      2015-01-28  1050501           ES   V   63            0.0   
8      2015-01-28  1050556           ES   H   23            0.0   
9      2015-01-28  1050536           ES   V   23            0.0   
10     2015-01-28  1050544           ES   H   25            0.0   
11     2015-01-28  1050713           ES   V   23            0.0   
12     2015-01-28  1050847           ES   V   22            0.0   
13     2015-01-28  1050811           ES   H   26            0.

[561786 rows x 43 columns]


In [20]:
user_data_all = user_data_all.drop(['sex','age','province_code','user_segmentation'],axis =1)
print(user_data_all)

       user_id  user_income
0        16074        53478
1        16117       465589
2        16181       134978
3        16308       175988
4        16339       197736
5        16450       202125
6        16465        37449
7        16565       187198
8        16571       210769
9        16579       196727
10       16630       187577
11       16656       156336
12       16861       118160
13       16972       186671
14       16979        96514
15       16990       244742
16       17013       430373
17       17015       192393
18       17035        53245
19       17075       180994
20       17094       186611
21       17100       377547
22       17121        78264
23       17176       213377
24       17218       235656
25       17269       132998
26       17300       107116
27       17360       137951
28       17398       212936
29       17434       188817
...        ...          ...
39667  1547587        80784
39668  1547621       125845
39669  1547636        96030
39670  1547674      

In [21]:
#将预测的收入数据填补进原有的数据集中
data = data.drop('user_income',axis=1)
data = pd.merge(data,user_data_all,how='left')

In [22]:
print(data)

             date  user_id user_country sex  age  new_user_type  \
0      2015-01-28  1050676           ES   V   31            0.0   
1      2015-01-28  1050662           ES   V   37            0.0   
2      2015-01-28  1050652           ES   H   31            0.0   
3      2015-01-28  1050489           ES   H   23            0.0   
4      2015-01-28  1050490           ES   V   23            0.0   
5      2015-01-28  1050497           ES   H   23            0.0   
6      2015-01-28  1050500           ES   V   24            0.0   
7      2015-01-28  1050501           ES   V   63            0.0   
8      2015-01-28  1050556           ES   H   23            0.0   
9      2015-01-28  1050536           ES   V   23            0.0   
10     2015-01-28  1050544           ES   H   25            0.0   
11     2015-01-28  1050713           ES   V   23            0.0   
12     2015-01-28  1050847           ES   V   22            0.0   
13     2015-01-28  1050811           ES   H   26            0.

In [23]:
train_data_new = data[data.origin == 'train']
test_data_new = data[data.origin == 'test']
train_data_new.to_csv('train_update.csv',index = 0)
test_data_new.to_csv('test_update.csv',index = 0)